In [1]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import re

In [2]:
df = unpkl('raw.pkl')

# Overall standard deviation

In [3]:
allthicknesses = [item for sublist in df.thickness for item in sublist]

In [4]:
thickstd = np.std(allthicknesses)
thickstd

0.08511185236696667

# Corrections

In [5]:
df = df.loc[~df.fname.str.contains('FOMU_Thickness_LML_7-26-23.xlsx')]

In [6]:
df.catalog.loc[df.catalog_manbran==' 1881c'] = "1881c"

In [7]:
idxs = df.loc[(df.catalog.str.contains('1984'))&(df.fname=='thickness_ll_box6_1_18.xlsx')].index
df = df.drop(idxs)

In [8]:
def catalog_change(df,pre,post):
    df.catalog.loc[df.catalog==pre] = post

In [9]:
catalog_change(df,'2856','2586')

In [10]:
catalog_change(df,'4179b','4791b')

In [11]:
catalog_change(df,'4199k','4911k')
catalog_change(df,'4199l','4911l')

In [12]:
catalog_change(df,'4973c','4793c')

In [13]:
catalog_change(df,'520q','5209q')

In [14]:
catalog_change(df,'5440a','5440')

In [15]:
df.catalog.loc[df.catalog_manbran=='2559_aa'] = '2259'

In [16]:
df.catalog.loc[df.catalog.str.contains("5606kk")] = [
    '5606kk',
    '5606ll',
    '5606mm',
    '5606nn'
]

In [17]:
df.catalog_manbran = df.catalog_manbran.apply(str) # unclear why I need to do this, avoids NaN mask error

In [18]:
df.catalog_manbran = df.catalog_manbran.fillna('')

In [19]:
df.catalog.loc[df.catalog_manbran.str.contains("2372dd_kt")] = '2372pp'

In [20]:
df.catalog.loc[df.catalog.str.contains("2523")&(df.fname.str.contains('binder5'))] = '2553'

In [21]:
df.catalog.loc[df.catalog.str.contains("3131")&(df.fname.str.contains('binder5'))] = '3113'

In [22]:
df.catalog.loc[(df.catalog=="27")&(df.fname.str.contains('binder5'))] = '2404'

In [23]:
df.catalog.loc[(df.catalog=="2166")&(df.fname.str.contains('binder5'))] = '2168'

In [24]:
df.catalog.loc[(df.catalog=="2512")&(df.fname.str.contains('binder5'))] = '2521'

In [25]:
df.catalog.loc[(df.catalog=="2304")&(df.fname.str.contains('binder2_7'))] = '2504'

In [26]:
df.catalog.loc[(df.catalog.str.contains("5408"))&(df.fname.str.contains('ll'))] = df.catalog.loc[(df.catalog.str.contains("5408"))&(df.fname.str.contains('ll'))].apply(lambda x: x.replace('5408','5409'))

In [27]:
df.catalog.loc[(df.catalog=='1433')&(df.fname.str.contains('ac'))] = '1443'

In [28]:
idxs = df.catalog.loc[(df.catalog.str.contains('2077'))&(df.fname=='thickness_ll_box2_1_7 (#2372).xlsx')].index
df = df.drop(idxs)

In [29]:
df.catalog.loc[(df.catalog.str.contains('5474'))&(df.fname=='thickness_ja_missingsample (version 1).xlsx')&(df.catalog_manbran.str.contains('_ib'))] = df.catalog.loc[(df.catalog.str.contains('5474'))&(df.fname=='thickness_ja_missingsample (version 1).xlsx')&(df.catalog_manbran.str.contains('_ib'))].apply(lambda x: x.replace('5474','5476'))

In [30]:
idxs = df.loc[df.catalog.str.contains('1880')&(df.fname=='Thicknes Measurement sheet_alam.xlsx')].index
df = df.drop(idxs)

In [31]:
bad_legacies = [
    '394','86','1875','334','4127','4125','2446','1021','402','2132','227','148','2557','191','1523','1474'
]

In [32]:
idxs = df.loc[(df.fname=='thickness_charts.xls')&(df.catalog.isin(bad_legacies))].index
df = df.drop(idxs)

# Modern dupes

### Enumeration of Cases

In [33]:
from itertools import combinations

In [34]:
exact_whitelist = ['5150','5155','4792e','4792n','5154','5036']
sb_whitelist = []

In [39]:
moddupes = df.catalog.value_counts().loc[lambda x: x > 1].index

In [40]:
for moddupe in moddupes:
    tmp = df[df.catalog==moddupe]
    
    # we exclude case where both Leah and I measured binder 4
    fnames = list(tmp.fname)
    duped_files = ['thickness_dc_binder4.xlsx','thickness_ll_binder4']
    
    # we exclude cases where the medians are within 1SD
    l = list(tmp.thickness.apply(np.median))
    medpairs = list(combinations(l, 2))
    meddiffs = [abs(item[0]-item[1]) for item in medpairs]
    
    if all([duped_file in fnames for duped_file in duped_files]):
        continue
    elif all([diff < thickstd for diff in meddiffs]):
        continue
    elif moddupe in exact_whitelist:
        continue
    else:
        print(moddupe)

In [43]:
df[['catalog','thickness','fname','catalog_manbran']].to_pickle('raw_dedupe.pkl')